In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from PIL import Image
from io import StringIO,BytesIO 
from scipy.spatial.distance import pdist
import cv2
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc 
from functools import reduce
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.set_device(0)
print (torch.cuda.current_device())

Loading faiss with AVX2 support.


0


In [7]:
trData = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/train.csv" , sep=',')
print('Dataset statistic, records: %d, fields: %d'%(trData.shape[0], trData.shape[1]))
print(trData.columns.values.tolist())

Dataset statistic, records: 369126, fields: 16
['path', 'view', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Airspace Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']


In [8]:
print(trData['view'].value_counts())

frontal    248236
lateral    120743
other         147
Name: view, dtype: int64


In [10]:
print(trData['Enlarged Cardiomediastinum'].value_counts())

-1.0    12407
 1.0     9844
 0.0     9796
Name: Enlarged Cardiomediastinum, dtype: int64


In [17]:
print(trData.head(5))

                                    path     view  No Finding  \
0  train/p10000032/s01/view1_frontal.jpg  frontal         1.0   
1  train/p10000032/s01/view2_lateral.jpg  lateral         1.0   
2  train/p10000032/s02/view1_frontal.jpg  frontal         1.0   
3  train/p10000032/s02/view2_lateral.jpg  lateral         1.0   
4  train/p10000032/s03/view1_frontal.jpg  frontal         1.0   

   Enlarged Cardiomediastinum  Cardiomegaly  Airspace Opacity  Lung Lesion  \
0                         NaN           NaN               NaN          NaN   
1                         NaN           NaN               NaN          NaN   
2                         NaN           NaN               NaN          NaN   
3                         NaN           NaN               NaN          NaN   
4                         NaN           NaN               NaN          NaN   

   Edema  Consolidation  Pneumonia  Atelectasis  Pneumothorax  \
0    NaN            NaN        NaN          NaN           NaN   
1    NaN  

In [11]:
teData = pd.read_csv("/data/fjsdata/physionet/MIMIC-CXR/valid.csv" , sep=',')
print('Dataset statistic, records: %d, fields: %d'%(teData.shape[0], teData.shape[1]))
print(teData.columns.values.tolist())

Dataset statistic, records: 2732, fields: 16
['path', 'view', 'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Airspace Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']


In [14]:
print(teData['view'].value_counts())

frontal    1759
lateral     972
other         1
Name: view, dtype: int64


In [15]:
print(teData['Enlarged Cardiomediastinum'].value_counts())

-1.0    79
 0.0    76
 1.0    59
Name: Enlarged Cardiomediastinum, dtype: int64


In [16]:
print(teData.head(5))

                                    path     view  No Finding  \
0  valid/p10228846/s01/view1_frontal.jpg  frontal         1.0   
1  valid/p10228846/s01/view2_lateral.jpg  lateral         1.0   
2  valid/p10228846/s02/view1_frontal.jpg  frontal         NaN   
3  valid/p10228846/s02/view2_lateral.jpg  lateral         NaN   
4  valid/p10248350/s01/view1_frontal.jpg  frontal         1.0   

   Enlarged Cardiomediastinum  Cardiomegaly  Airspace Opacity  Lung Lesion  \
0                         NaN           NaN               NaN          NaN   
1                         NaN           NaN               NaN          NaN   
2                         NaN           NaN               1.0          NaN   
3                         NaN           NaN               1.0          NaN   
4                         NaN           NaN               NaN          NaN   

   Edema  Consolidation  Pneumonia  Atelectasis  Pneumothorax  \
0    NaN            NaN        NaN          NaN           NaN   
1    NaN  